# Load Data

In [1]:
!wget https://storage.googleapis.com/gresearch/dakshina/dakshina_dataset_v1.0.tar
!tar -xvf  'dakshina_dataset_v1.0.tar'

--2021-05-17 10:57:02--  https://storage.googleapis.com/gresearch/dakshina/dakshina_dataset_v1.0.tar
Resolving storage.googleapis.com (storage.googleapis.com)... 64.233.184.128, 74.125.133.128, 74.125.140.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|64.233.184.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2008340480 (1.9G) [application/x-tar]
Saving to: ‘dakshina_dataset_v1.0.tar’

dakshina_dataset_v1 100%[===================>]   1.87G  35.3MB/s    in 17s     

2021-05-17 10:57:19 (114 MB/s) - ‘dakshina_dataset_v1.0.tar’ saved [2008340480/2008340480]

dakshina_dataset_v1.0/bn/
dakshina_dataset_v1.0/bn/lexicons/
dakshina_dataset_v1.0/bn/lexicons/bn.translit.sampled.test.tsv
dakshina_dataset_v1.0/bn/lexicons/bn.translit.sampled.train.tsv
dakshina_dataset_v1.0/bn/lexicons/bn.translit.sampled.dev.tsv
dakshina_dataset_v1.0/bn/native_script_wikipedia/
dakshina_dataset_v1.0/bn/native_script_wikipedia/bn.wiki-filt.valid.text.shuf.txt.gz

# load and create model

In [2]:
%pip install wandb -q
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping
import wandb
from wandb.keras import WandbCallback

     |████████████████████████████████| 1.8MB 11.8MB/s 
     |████████████████████████████████| 133kB 39.0MB/s 
     |████████████████████████████████| 174kB 35.2MB/s 
     |████████████████████████████████| 102kB 10.7MB/s 
     |████████████████████████████████| 71kB 8.6MB/s 


In [38]:
input_token_index = None
target_token_index = None
MAX_LEN_input = None
MAX_LEN_target = None
num_encoder_tokens = 30
num_decoder_tokens = 70
input_tokenizer = None
target_tokenizer = None

In [39]:
def tokenize(data,vocab_size):
  tokenizer = Tokenizer(num_words=vocab_size,char_level=True)
  tokenizer.fit_on_texts(data)
  temp=tokenizer.texts_to_sequences(data)
  # print(data[:3])
  # print(temp[:3])
  dictionary = tokenizer.word_index
  return temp , dictionary , tokenizer

In [40]:
def load_and_preprocess():
  global input_token_index , target_token_index , MAX_LEN_input , MAX_LEN_target ,num_decoder_tokens,num_encoder_tokens , input_tokenizer , target_tokenizer
  data_path = 'dakshina_dataset_v1.0/gu/lexicons/gu.translit.sampled.train.tsv'
  input_texts = []
  target_texts = []
  with open(data_path, "r", encoding="utf-8") as f:
    lines = f.read().split("\n")
  for line in lines[:-1]:
    temp = line.split('\t')
    input_text, target_text = temp[1],temp[0] 
    target_text = "\t" + target_text + "\n"
    input_text = input_text+"\n"

    input_texts.append(input_text)
    target_texts.append(target_text)
  
  MAX_LEN_input = max([len(txt) for txt in input_texts])
  MAX_LEN_target = max([len(txt) for txt in target_texts])

  # tokenize
  encoder_input , input_token_index , input_tokenizer = tokenize(input_texts , num_encoder_tokens)
  decoder_input , target_token_index, target_tokenizer = tokenize(target_texts , num_decoder_tokens) 

  # padding
  encoder_input_data = pad_sequences(encoder_input, maxlen=MAX_LEN_input, dtype='int32', padding='post', truncating='post',value= input_token_index["\n"])
  decoder_input_data = pad_sequences(decoder_input, maxlen=MAX_LEN_target, dtype='int32', padding='post', truncating='post',value=target_token_index["\n"])

  decoder_target_data = np.zeros((len(input_texts), MAX_LEN_target, num_decoder_tokens), dtype="float32")
  for i,  target_text in enumerate(target_texts):
    for t, char in enumerate(target_text):
      if t > 0:
        decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
    decoder_target_data[i, t:, target_token_index["\n"]] = 1.0

  return encoder_input_data , decoder_input_data, decoder_target_data



In [45]:
def load_val_data(data_path = 'dakshina_dataset_v1.0/gu/lexicons/gu.translit.sampled.dev.tsv'):
  global input_token_index , target_token_index , MAX_LEN_input , MAX_LEN_target ,num_decoder_tokens,num_encoder_tokens , input_tokenizer , target_tokenizer
  
  input_texts = []
  target_texts = []
  with open(data_path, "r", encoding="utf-8") as f:
    lines = f.read().split("\n")
  for line in lines[:-1]:
    temp = line.split('\t')
    target_text , input_text = temp[0],temp[1] 
    target_text = "\t" + target_text + "\n"
    input_text = input_text+"\n"

    input_texts.append(input_text)
    target_texts.append(target_text)
  
  # tokenize
  encoder_input  = input_tokenizer.texts_to_sequences(input_texts)
  decoder_input  = target_tokenizer.texts_to_sequences(target_texts) 

  # padding
  encoder_input_data = pad_sequences(encoder_input, maxlen=MAX_LEN_input, dtype='int32', padding='post', truncating='post',value= input_token_index["\n"])
  decoder_input_data = pad_sequences(decoder_input, maxlen=MAX_LEN_target, dtype='int32', padding='post', truncating='post',value=target_token_index["\n"])

  decoder_target_data = np.zeros((len(input_texts), MAX_LEN_target, num_decoder_tokens), dtype="float32")
  for i,  target_text in enumerate(target_texts):
    for t, char in enumerate(target_text):
      if char == 'ૠ':
        char = 'ઋ'
      if t > 0:
        decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
    decoder_target_data[i, t:, target_token_index["\n"]] = 1.0

  return encoder_input_data , decoder_input_data, decoder_target_data

# load_val_data()

In [24]:
from tensorflow import keras
from tensorflow.keras.layers import Dense,LSTM,Input,GRU,SimpleRNN,Dropout,Embedding

In [42]:
def create_model(m_name="LSTM",n_e_layers=1,n_d_layers=1,latent_dim = 100,embedding_size = 16,dropout = 0 , recurrent_dropout = 0):
  keras.backend.clear_session()
  # num_encoder_tokens = 30
  # num_decoder_tokens = 50
  # Define an input sequence and process it.
  input1 = Input(shape=(None,),name= "input_1")
  encoder_inputs = Embedding(input_dim = num_encoder_tokens, output_dim = embedding_size)(input1)

  encoder = globals()[m_name](latent_dim,dropout=dropout,recurrent_dropout = recurrent_dropout, return_state=True,return_sequences=True)
  e_o = encoder(encoder_inputs)
  prev = e_o
  for i in range(1,n_e_layers):
    e = globals()[m_name](latent_dim, dropout=dropout,recurrent_dropout = recurrent_dropout,return_state=True,return_sequences=True)
    e_o = e(prev[0])
    prev = e_o
  
  input2 = Input(shape=(None,),name="input_2")
  decoder_inputs = Embedding(input_dim = num_decoder_tokens, output_dim = embedding_size)(input2)
  d_l = globals()[m_name](latent_dim,dropout=dropout,recurrent_dropout = recurrent_dropout, return_sequences=True, return_state=True)(decoder_inputs,initial_state = e_o[1:])
  p_d = d_l[0]
  for i in range(1,n_d_layers):
    d_l = globals()[m_name](latent_dim,dropout=dropout,recurrent_dropout = recurrent_dropout, return_state=True, return_sequences=True)(p_d,initial_state = e_o[1:])
    p_d = d_l[0]

  
  decoder_dense = Dense(num_decoder_tokens, activation="softmax")
  decoder_outputs = decoder_dense(d_l[0])

  # Define the model that will turn
  # `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
  model = keras.Model([input1,input2], decoder_outputs)

  return model


# sweep

In [50]:
def train():
  run = wandb.init()
  c = run.config
  name = "model_"+c.model+"_o_"+c.optimizer+"_el_"+str(c.encoder_layers)+"_dl_"+str(c.decoder_layers)+"_hs_"+str(c.hidden_size)+"_em_"+str(c.embedding_size)+"_d_"+str(c.dropout)
  run.name = name
  print(name)
  batch_size = 128
  epochs = 20

  encoder_input_data,decoder_input_data ,decoder_target_data = load_and_preprocess()
  val_encoder_input_data,val_decoder_input_data ,val_decoder_target_data = load_val_data()
  es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=3)

  model = create_model(c.model,c.encoder_layers,c.decoder_layers,c.hidden_size,c.embedding_size,c.dropout,0)
  model.compile(optimizer=c.optimizer, loss="categorical_crossentropy", metrics=["accuracy"])
  model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=batch_size,
    epochs=epochs,
    validation_data=([val_encoder_input_data, val_decoder_input_data],val_decoder_target_data),
    # validation_split=0.2,
    callbacks=[WandbCallback(),es]
  )
  temp = model.predict([val_encoder_input_data, val_decoder_input_data]).argmax(axis=-1)
  val_word_acc = sum((temp[:,:-1] == val_decoder_input_data[:,1:]).all(axis=-1)) / len(val_encoder_input_data)
  # print(val_word_acc)

  temp = model.predict([encoder_input_data,decoder_input_data]).argmax(axis=-1)
  train_word_acc = sum((temp[:,:-1] == decoder_input_data[:,1:]).all(axis=-1)) / len(encoder_input_data)
  # print(train_word_acc)

  wandb.log({"val_word_acc" : round(val_word_acc,4) , "train_word_acc" : round(train_word_acc,4)})
  return

In [48]:
sweep_config={
    'method' : 'random' ,
    'metric' : { 'name' : 'val_word_acc' , 'goal' : 'maximize' } ,
    'parameters' : {
        'model' : { 'values' : ['LSTM','GRU','SimpleRNN'] },
        'dropout' : { 'values' : [0.1,0.2]},
        'embedding_size' : {'values' : [64,128]},
        'encoder_layers' : {'values' : [1,2,3]},
        'decoder_layers' : {'values' : [1,2,3]},
        'hidden_size' : {'values' : [128,256,512]},
        'optimizer' : {'values' : ['rmsprop' ,'adam']}
    }
}


In [51]:

sweepid = wandb.sweep(sweep_config,project="DL_A3_Q2_final",entity ="sonagara")
wandb.agent(sweepid,train)

<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Create sweep with ID: u6pg8nh7
Sweep URL: https://wandb.ai/sonagara/DL_A3_Q1_testing1/sweeps/u6pg8nh7


...

epoch,0
loss,1.1351
accuracy,0.70639
val_loss,0.92348
val_accuracy,0.74362
_runtime,135
_timestamp,1621252192
_step,1
best_val_loss,0.92348
best_epoch,0
val_word_acc,0.0001


epoch,▁
loss,▁
accuracy,▁
val_loss,▁
val_accuracy,▁
_runtime,▁█
_timestamp,▁█
_step,▁█
val_word_acc,▁
train_word_acc,▁


wandb: Agent Starting Run: c5yx5vew with config:
wandb: 	decoder_layers: 1
wandb: 	dropout: 0.1
wandb: 	embedding_size: 128
wandb: 	encoder_layers: 2
wandb: 	hidden_size: 128
wandb: 	model: LSTM
wandb: 	optimizer: rmsprop


model_LSTM_o_rmsprop_el_2_dl_1_hs_128_em_128_d_0.1
208/823 [======>.......................] - ETA: 22s - loss: 1.4987 - accuracy: 0.6681

wandb: Ctrl + C detected. Stopping sweep.


# best model

In [46]:
def train_best():
  m_name = "GRU"
  encoder_layers = 2
  decoder_layers = 2
  latent_dim = 512
  embedding_size = 128
  dropout = 0.35
  batch_size = 128
  recurrent_dropout = 0  # 0 to use cudnnlstm which is faster than lstm

  encoder_input_data,decoder_input_data ,decoder_target_data = load_and_preprocess()
  val_encoder_input_data,val_decoder_input_data ,val_decoder_target_data = load_val_data()
  

  model = create_model(m_name,encoder_layers,decoder_layers,latent_dim,embedding_size,dropout,recurrent_dropout)
  # model = keras.models.load_model("s2s")
  model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

  es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=3)

  model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=batch_size,
    epochs=20,
    # validation_split = 0.2,
    validation_data=([val_encoder_input_data, val_decoder_input_data],val_decoder_target_data),
    callbacks=[es]
  )

  temp = model.predict([val_encoder_input_data, val_decoder_input_data]).argmax(axis=-1)
  val_word_acc = sum((temp[:,:-1] == val_decoder_input_data[:,1:]).all(axis=-1)) / len(val_encoder_input_data)
  print(val_word_acc)

  temp = model.predict([encoder_input_data,decoder_input_data]).argmax(axis=-1)
  train_word_acc = sum((temp[:,:-1] == decoder_input_data[:,1:]).all(axis=-1)) / len(encoder_input_data)
  print(train_word_acc)
  


  # Save model
  model.save("s2s")
  # print(test_acc("LSTM",100,1,2))
train_best()

Epoch 1/20
823/823 [==============================] - 79s 88ms/step - loss: 1.1875 - accuracy: 0.7059 - val_loss: 0.6406 - val_accuracy: 0.8110
Epoch 2/20
823/823 [==============================] - 72s 87ms/step - loss: 0.5114 - accuracy: 0.8462 - val_loss: 0.2604 - val_accuracy: 0.9188
Epoch 3/20
823/823 [==============================] - 72s 87ms/step - loss: 0.1858 - accuracy: 0.9425 - val_loss: 0.2012 - val_accuracy: 0.9382
Epoch 4/20
823/823 [==============================] - 72s 87ms/step - loss: 0.1074 - accuracy: 0.9666 - val_loss: 0.1968 - val_accuracy: 0.9417
Epoch 5/20
823/823 [==============================] - 72s 87ms/step - loss: 0.0767 - accuracy: 0.9759 - val_loss: 0.1964 - val_accuracy: 0.9431
Epoch 6/20
823/823 [==============================] - 72s 87ms/step - loss: 0.0607 - accuracy: 0.9808 - val_loss: 0.2083 - val_accuracy: 0.9430
Epoch 7/20
823/823 [==============================] - 72s 88ms/step - loss: 0.0514 - accuracy: 0.9833 - val_loss: 0.2158 - val_accuracy:

INFO:tensorflow:Assets written to: s2s/assets


INFO:tensorflow:Assets written to: s2s/assets


In [ ]:
import re
def load_test_data():
  global input_token_index , MAX_LEN_input , num_encoder_tokens , input_tokenizer
  data_path = 'dakshina_dataset_v1.0/gu/lexicons/gu.translit.sampled.test.tsv'
  input_texts=[]
  target_texts = []
  s = r'ૠ'
  # input_characters = set()
  with open(data_path,"r",encoding="utf-8") as f:
    lines = f.read().split("\n")
  for line in lines[:-1]:
    temp = line.split("\t")
    input_text , target_text = temp[0] ,temp[1]
    target_text = re.sub(s,'ઋ',target_text)
    input_text = input_text + "\n"
    
    input_texts.append(input_text)
    target_texts.append(target_text)
  
  encoder_input = input_tokenizer.texts_to_sequences(input_texts)
  encoder_input_data = pad_sequences(encoder_input, maxlen=MAX_LEN_input, dtype='int32', padding='post', truncating='post',value= input_token_index["\n"])
  print(encoder_input_data.shape)
  
  return encoder_input_data , target_texts
# load_test_data()

In [ ]:
def enc_dec(m_name="LSTM" ,latent_dim = 100, n_e_layers = 1,n_d_layers = 1):
  model = keras.models.load_model("s2s")
 
  if (n_e_layers == 1):
    l_name = ""
  else:
    l_name = "_"+str(n_e_layers-1)

  if (m_name == "SimpleRNN"):
    n_name = "simple_rnn"
  else:
    n_name = m_name

  # encoder
  encoder_inputs = model.input[0]
  encoder_outputs, *encoder_states = model.get_layer(n_name.lower()+l_name).output  # last encoding layer
  encoder_model = keras.Model(encoder_inputs, encoder_states)
  # encoder_model.summary()

  # decoder
  decoder_inputs = model.input[1] 
  decoder_embed = model.get_layer("embedding_1")(decoder_inputs)
  decoder_state_input_h = keras.Input(shape=(latent_dim,))
  decoder_state_input_c = keras.Input(shape=(latent_dim,))
  decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
  d_o = model.get_layer(n_name.lower()+"_"+str(n_e_layers))(decoder_embed, initial_state=decoder_states_inputs)
  for i in range(1+n_e_layers,n_d_layers+n_e_layers):
    d_o = model.get_layer(n_name.lower()+"_"+str(i))(d_o[0], initial_state=decoder_states_inputs)
  decoder_outputs = d_o[0]
  decoder_states = d_o[1:]
  decoder_dense = model.get_layer("dense")
  decoder_outputs = decoder_dense(decoder_outputs)
  decoder_model = keras.Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)
  # decoder_model.summary()

  return encoder_model , decoder_model
# enc_dec("LSTM",256,3,2)

In [ ]:

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())


def decode_sequence(input_seq,encoder_model,decoder_model):
    global num_decoder_tokens , target_token_index , reverse_target_char_index , MAX_LEN_target
    # Encode the input as state vectors.

    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = target_token_index["\t"]

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ""
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if sampled_char == "\n" or len(decoded_sentence) > MAX_LEN_target:
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = target_token_index[sampled_char]

        # Update states
        states_value = [h, c]
    return decoded_sentence


In [ ]:
def test_acc(m_name="LSTM" ,latent_dim = 100, n_e_layers = 1,n_d_layers = 1):
  decode_results = []
  x_test , y_test = load_test_data()
  enc ,dec = enc_dec(m_name , latent_dim , n_e_layers ,n_d_layers)
  score = 0 
  print(len(y_test))

  for seq_index in range(len(y_test)):
    
    input_seq = x_test[seq_index : seq_index + 1]
    decoded_sentence = decode_sequence(input_seq,enc,dec)
    decode_results.append(decoded_sentence)
    if (y_test[seq_index] == decoded_sentence):
      score += 1
    # print("-")
    # print("Input sentence:", y_test[seq_index])
    # print("Decoded sentence:", decoded_sentence)
  print(score/len(y_test))
  return score/len(y_test)

In [ ]:
print(test_acc("LSTM",256,3,2))

(10373, 19)
10373


Exception ignored in: <function WeakKeyDictionary.__init__.<locals>.remove at 0x7f5f2d1b10e0>
Traceback (most recent call last):
  File "/usr/lib/python3.7/weakref.py", line 358, in remove
    def remove(k, selfref=ref(self)):
KeyboardInterrupt


KeyboardInterrupt: ignored

In [33]:
# model = keras.models.load_model("s2s")
encoder_input_data,decoder_input_data ,decoder_target_data = load_and_preprocess()
print(target_token_index)

{'\t': 1, '\n': 2, 'ા': 3, 'ન': 4, 'ર': 5, '્': 6, 'ી': 7, 'વ': 8, 'મ': 9, 'ં': 10, 'ક': 11, 'ે': 12, 'િ': 13, 'સ': 14, 'ત': 15, 'ો': 16, 'લ': 17, 'ય': 18, 'પ': 19, 'ુ': 20, 'દ': 21, 'ગ': 22, 'ટ': 23, 'જ': 24, 'બ': 25, 'શ': 26, 'ડ': 27, 'હ': 28, 'થ': 29, 'ણ': 30, 'ચ': 31, 'ળ': 32, 'ધ': 33, 'ઓ': 34, 'ૂ': 35, 'ભ': 36, 'ખ': 37, 'અ': 38, 'ફ': 39, 'ષ': 40, 'આ': 41, 'એ': 42, 'ઉ': 43, 'ઇ': 44, 'ઈ': 45, 'છ': 46, 'ઘ': 47, 'ઝ': 48, 'ઠ': 49, 'ૃ': 50, 'ઢ': 51, 'ૈ': 52, 'ઞ': 53, 'ૌ': 54, 'ઊ': 55, 'ૉ': 56, 'ઃ': 57, 'ઔ': 58, 'ઋ': 59, 'ઐ': 60, 'ૅ': 61, 'ઑ': 62, 'ઁ': 63, 'ઍ': 64, 'ૐ': 65, '઼': 66}


In [12]:
a = model.predict([encoder_input_data,decoder_input_data]).argmax(axis=-1)
# print(a.argmax(axis=-1)[0][:-1])
# print(decoder_input_data[0:1][0][1:])
# if (a.argmax(axis=-1)[0][:-1].all() == decoder_input_data[0:1][0][1:].all()):
#   print("hi")

In [13]:
a.shape

(10373, 24)

In [14]:
temp = a
temp.shape
print(len(temp[0][:-1]))

23


In [ ]:
print(temp[:,:-1][1])

[ 1  5 13  1 17  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3]


In [15]:
len(decoder_input_data[:,1:][1])

23

In [ ]:
sum((temp[:,:-1] == decoder_input_data[:,1:]).all(axis=-1)) / len(encoder_input_data)


0.16890002892123782

In [16]:
len(encoder_input_data)

10373